In [9]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import torch

In [10]:
#set parameters
vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000

In [11]:
generate = {}

In [12]:
device = torch.device("cuda")

In [14]:
for pi in pi_vals:
  ADE = []
  AIE = []
  ASDE = []

  for i in trange(1000):
    G = nx.random_regular_graph(adjacencies, vertices)
    A = nx.adjacency_matrix(G).todense()
    A_gpu = torch.tensor(A).float().to(device)
    A_2 = torch.matmul(A_gpu, A_gpu).cpu().numpy()
    A_2 = np.where(A_2 > 0, 1, 0)
    np.fill_diagonal(A_2, 0)
    A_2 = A_2 - A # second degree neighbors
    W = np.random.binomial(1, pi, vertices)
    noise = np.random.normal(0, 1, vertices)
    Y = (W @ A_2)/(3 * np.sum(A_2, axis=1)) + (W @ A)/600 + W * (1/2) + noise # reexamine if the 1/300 coeff is still valid

    average_direct_effect = (np.dot(Y,W))/sum(W) - (np.dot(Y, (1-W)))/sum((1-W))
    ADE.append(average_direct_effect)
    average_indirect_effect = ((np.dot(A @ Y, W))/sum(W) - (np.dot(A @ Y, (1-W)))/sum(1-W))/adjacencies
    avg_second_deg = ((np.dot(A_2 @ Y, W))/sum(W) - (np.dot(A_2 @ Y, (1-W)))/sum(1-W))/(vertices - adjacencies - 1)
    ASDE.append(avg_second_deg)
    AIE.append(average_indirect_effect)

  str1 = str(pi)+" ADE"
  str2 = str(pi)+" AIE"
  str3 = str(pi)+" ASDE"
  generate[str1] = ADE
  generate[str2] = AIE
  generate[str3] = ASDE


100%|██████████| 1000/1000 [07:52<00:00,  2.12it/s]


In [15]:
df_g = pd.DataFrame.from_dict(generate)
df_g.head()

,0.25 ADE,0.25 AIE,0.25 ASDE,0.5 ADE,0.5 AIE,0.5 ASDE,0.75 ADE,0.75 AIE,0.75 ASDE
0,0.319413,0.000249,-0.000386,0.485178,-0.002492,-0.000273,0.525663,0.002774,-0.000904
1,0.541863,0.005782,-0.001255,0.496550,-0.004259,-0.000083,0.480767,-0.009518,0.000543
2,0.502400,-0.003778,-0.000131,0.526142,0.010287,-0.001723,0.560576,0.016883,-0.002505
3,0.512405,0.003000,-0.000905,0.384376,-0.002679,-0.000127,0.454678,-0.013102,0.000944
4,0.540185,0.006867,-0.001364,0.463450,-0.007670,0.000343,0.483207,0.022120,-0.003002


In [16]:
df_g.to_csv('11_26_23_1st2nd_Degree_Model_1_1316reverse_505_Data.csv')